# Vectores, operaciones vectoriales básicas y Universal functions



Como cientificos de datos en algunos casos requeriremos crear nuestras propias funciones y es recomendable que se apeguen a las ufuncs, es decir que se puedan aplicar independiente de la dimensión del objeto al que se aplican, que soporten broadcasting y muy importante **que su definición evite ciclos y use operaciones vectorizadas y otras ufuncs** .

In [1]:
import numpy as np

## Ejercicio 1
En data science y programación científica es común necesitar vectores con valores dentro de cierto intervalo, pero con sub-intervalos internos con diference espaciamiento ,por ejemplo:

[0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ]
       
El primer sub-intervalo incrementa de 0.1 en 0.1 , el segundo de 0.01 en 0.01 , y el tercero de 0.1 en 0.1 nuevamente.

Usando unicamente NumPy crea un vector de este tipo con el nombre xs .

**nota**: no es permitido usar ciclos, debe realizarse con operaciones "vectorizadas" de numpy

In [2]:
## tu codigo aqui (~ 4 lineas de codigo)
xs = np.hstack((np.linspace(0,0.4,5),np.linspace(0.5,0.59,10),np.linspace(0.6,1,5)))
xs

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ])

## Ejercicio 2
Suavizado de curvas a través de promedios de n-puntos(medias móviles):

<img src="https://www.luisllamas.es/wp-content/uploads/2017/03/arduino-filtro-media-movil-ventana-5.png">

En el mundo la data comunmente posee "ruido" por lo cual muchas veces antes de hacer análisis o modelado se necesita "suavizar" las curvas para reducir este ruido, una técnica muy sencilla es  el suavizado por promedio de n-puntos(también llamado media móvil), esto significa que para cada punto Xn obtenemos una versión transformada(suavizada) que consiste en promediar n puntos cercanos a el, por ejemplo para n= 3 tenemos que:

$$XS_{n}  =  \frac{X_{n-1}+X_{n}+X_{n+1}}{3}$$

In [3]:
# las siguientes 3 lineas generan un conjunto de datos que se comportan segun una onda senoidal pero tienen
# ruido por lo cual vamos a suavizar usando promedio de 3 puntos
# en este ejercicio los generamos manualmente pero pensemos que son datos que pudieron ser generados con algún
# instrumento como un sensor, o bien estar almacenados en una base de datos
ruido = 0.1*np.random.randn(15) #el ruido comunmente se debe a aleatoriedad o captura no exacta de info.
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

#xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()

for n in range(1,len(x)-1):
    xs[n] = (x[n-1] + x[n] + x[n+1])/3
    
print(x)
print(xs)

[ 0.15002364  0.3495207   0.73824892  0.96914005  0.95371951  0.79042113
  0.48094297  0.11116648 -0.35448183 -0.65097061 -0.998631   -0.96839141
 -0.77987504 -0.39437041 -0.02160723]
[ 0.15002364  0.41259775  0.68563656  0.88703616  0.9044269   0.74169454
  0.46084353  0.07920921 -0.29809532 -0.66802781 -0.87266434 -0.91563249
 -0.71421229 -0.39861756 -0.02160723]


En este caso usamos un ciclo  para ejemplificar pero ya hemos mencionado que debemos evitarlos siempre que sea posible y usar operaciones "vectorizadas" ya que es mucho mas rápido al utilizar  código pre-compilado de C, en este ejercicio debemos remplazar el calculo de xs para hacerlo de manera "vectorizada" y eliminar el ciclo for. 

**tip**: usar slicing

In [4]:
ruido = 0.1*np.random.randn(15) 
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

# xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()

## tu codigo aqui (~ 1 linea):
xs = np.hstack((x[0], (xs[slice(2,len(xs),1)] + xs[slice(1,len(xs)-1,1)] + xs[slice(0,len(xs)-2,1)])/3, x[-1]))

print(x)
print(xs)


[ 0.01742038  0.39284106  0.7451717   0.9840965   0.99504191  0.76573902
  0.43570504  0.0994905  -0.39008767 -0.81606676 -0.92471697 -0.96144036
 -0.76916791 -0.40756506 -0.00415892]
[ 0.01742038  0.38514438  0.70736975  0.90810337  0.91495914  0.73216199
  0.43364485  0.04836929 -0.36888798 -0.71029047 -0.90074136 -0.88510841
 -0.71272444 -0.39363063 -0.00415892]


## Ejercicio 3

NumPy posee logaritmos naturales , base 2 y base 10 , crear una *ufunc* que calcule el logaritmo en cualquier base n dada, la funcion debe recibir el tensor al cual calcular el logaritmo elemento por elemento y la base del logaritmo a calcular, por ejemplo:

`def logaritmo_n(x,n):`

In [5]:
## tu codigo aqui:
def logN(tensor,n):
    return np.log(tensor)/np.log(n)

t = np.array(([10,100,1000],
            [9, 27, 81]))
logN(t,3)

array([[2.09590327, 4.19180655, 6.28770982],
       [2.        , 3.        , 4.        ]])

## Ejercicio 4
Para calcular la información de una distribución de probabilidad necesitamos calcular el producto entre la información de cada posible $x$ y su probabilidad, cada uno de estos elementos nos indica cuanta incerteza aporta cada $x$ a la entropía de la distribución. 

Para completar este ejercicio hay que crear una función que calcule lo descrito anteriormente, debe recibir un vector representando a la dist. de probabilidad y devoler otro vector con el termino de entropía para cada $x$.

Por ejemplo:

`calcular_entropia([0.25,0.75])`

Debe resultar en :

$[0.5    ,    0.31127812]$

**Nota**: la entropía es la suma sobre estos valores, en este caso no calculamos la entropía,unicamente los termimos de su sumatoria. La entropía es:

$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

In [8]:
## tu codigo aqui (~ 2 lineas de codigo)
def calcEnt(distProb):
    return -logN(distProb,2)*distProb

calcEnt([0.25,0.75])

array([0.5       , 0.31127812])

## Ejercicio 5

A partir de la información en un data warehouse se ha generado un modelo de datos, el valor de los clientes en el tiempo se ha determinado un modelo predictivo  no lineal que estima cuanta ganancia ha generado cierto cliente en 5 años en base a lo que ha gastado en su primer mes como cliente. 

El modelo tiene la forma:

$$g(x) = (x^{3} + 2x^{2} + e^{0.0001x} -\sqrt{2x})/50$$

Crear una función que calcule la ganancia para valores de gasto de nuevos clientes:

In [9]:
def g(x):
    ganancia = (np.power(x,3) + 2.0*(np.power(x,2.0))+np.exp(0.0001*x) - np.sqrt(2.0*x))/50
    return ganancia
    
gastos = np.array([100,50,25])
ganancia = g(gastos)

print("Ganancias",ganancia)

Ganancias [20399.73735829  2599.82010025   337.37862871]


Se ha determinado también que hay costos asociados a la comunicación y relación con los clientes(llamadas,mensajes electrónicos,etc) y que estos también tienen alta correlación con el monto que estos gastan en su primer més siguiendo el modelo:
$$c(x) = (x^{2} + x + log (0.0001x)-\sqrt{0.54x})/100$$

Cree una función para calcular este modelo sobre valores de de gasto de nuevos clientes , y luego cree una función  **ganancia_neta(x,g,c)** donde:
* x = vector conteniendo los valores de gasto del primer mes para nuevos clientes.
* g = función que estima la ganancia en función de x
* c = función que estima el costo en función de x

La función realiza el calculo simple **g(x) - c(x)** , ya que es posible que los modelos cambien en el tiempo es común recibir como parámetro el modelo o modelos hijos de un modelo mas grande , en este caso eso se logra recibiendo como parámetro las funciones de ganancia y costo.

In [16]:
## tu codigo aqui :
def g(x):
    return (np.power(x,3) + 2*(np.power(x,2))+ np.exp(0.0001*x) - np.sqrt(2.0*x))/50
def c(x):
    return (np.power(x,2) + x + np.log(0.0001*x) - np.sqrt(0.54*x))/100

def GananciaNeta(x,g,c):
    return g(x)-c(x)

x = np.array([100,50,25])

GananciaNeta(x,g,c)


array([20298.85689469,  2574.42504495,   330.9752857 ])

# Funciones miscelaneas(agregación y estadistica descriptiva básica) 

Ya hemos trabajado algunas funciones de NumPy pero han sido funciones "elementwise" cuyo vector resultante tiene el mismo tamaño que el vector de entrada, existen muchas otras funciones que utilizamos en casos en los que necesitamos agregar,resumir cierto vector,incluyendo en este tipo de funciones algunas de estadistica descriptiva básica.

## Ejercicio 6
Usando argmax crear una función `mode(x)` para calcular la moda de un vector $x$, esta función debe devolver una tupla de 2 elementos de la forma:

(valor,conteo)

**tip** investigar y auxiliarse de otras funciones de numpy y np.argmax, no olvidar trabajar todo vectorizado, sin ciclos.

In [19]:
def mode(x):
    ## tu codigo aqui (~ 3 lineas de codigo):
    unique, count = np.unique(x,return_counts=True)
    r = np.asarray((unique,count))
    return (r[0][np.argmax(r[1])],r[1][np.argmax(r[1])])

a = [1,2,3,5,6,7,8,9,0,5,4,3,2,1,2,4,6,7,8,7,6,7,6,5]
print(mode(a))

(6, 4)


## Ejercicio 7
Crear una función para calcular la entropía  de una distribución de probabilidad representada como un vector.
$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

**Nota**: a diferencia del ejercicio anterior donde solo calculamos los términos de la entropía, en este caso  si calculamos el valor completo de la entropía, que como mencionamos es una medida de incerteza en una distribución de probabilidad.

**recordatorio**: no usar ciclos solo operaciones vectorizadas.

In [20]:
## tu codigo aqui (~ 3 lineas de codigo)
def Ent(distProb):
    return -np.dot(logN(distProb,2),distProb)

Ent([0.25,0.75])

0.8112781244591328